# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [13]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [8]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',con=engine)
X = df.message 
y = df.iloc[:,4:]
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    tokens=word_tokenize(text)
    lemmatizer=WordNetLemmatizer()
    clean_tokens=[]
    
    for tok in tokens:
        clean_tok=lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
#train classifier
pipeline.fit(X_train, y_train)

In [14]:
# predict on test data
y_pred = pipeline.predict(X_test)

related              precision    recall  f1-score   support

          0       0.63      0.33      0.43      1534
          1       0.81      0.94      0.87      4968
          2       0.69      0.17      0.28        52

avg / total       0.77      0.79      0.77      6554

request              precision    recall  f1-score   support

          0       0.89      0.98      0.93      5473
          1       0.81      0.38      0.52      1081

avg / total       0.88      0.88      0.87      6554

offer              precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.71      0.87      0.79      3820
          1       0.74      0.51      0.61      2734

avg / total       0.73      0.72      0.71      6554

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


medical_products              precision    recall  f1-score   support

          0       0.95      1.00      0.97      6221
          1       0.65      0.07      0.13       333

avg / total       0.94      0.95      0.93      6554

search_and_rescue              precision    recall  f1-score   support

          0       0.97      1.00      0.99      6356
          1       0.95      0.11      0.19       198

avg / total       0.97      0.97      0.96      6554

security              precision    recall  f1-score   support

          0       0.98      1.00      0.99      6418
          1       0.33      0.01      0.01       136

avg / total       0.97      0.98      0.97      6554

military              precision    recall  f1-score   support

          0       0.97      1.00      0.98      6339
          1       0.73      0.07      0.14       215

avg / total       0.96      0.97      0.96      6554

child_alone              precision    recall  f1-score   support

          0       1.0

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
for n, column in enumerate(y):
    print(column, classification_report(y_test.iloc[:,n],y_pred[:,n]))

related              precision    recall  f1-score   support

          0       0.63      0.33      0.43      1534
          1       0.81      0.94      0.87      4968
          2       0.69      0.17      0.28        52

avg / total       0.77      0.79      0.77      6554

request              precision    recall  f1-score   support

          0       0.89      0.98      0.93      5473
          1       0.81      0.38      0.52      1081

avg / total       0.88      0.88      0.87      6554

offer              precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.71      0.87      0.79      3820
          1       0.74      0.51      0.61      2734

avg / total       0.73      0.72      0.71      6554

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


medical_products              precision    recall  f1-score   support

          0       0.95      1.00      0.97      6221
          1       0.65      0.07      0.13       333

avg / total       0.94      0.95      0.93      6554

search_and_rescue              precision    recall  f1-score   support

          0       0.97      1.00      0.99      6356
          1       0.95      0.11      0.19       198

avg / total       0.97      0.97      0.96      6554

security              precision    recall  f1-score   support

          0       0.98      1.00      0.99      6418
          1       0.33      0.01      0.01       136

avg / total       0.97      0.98      0.97      6554

military              precision    recall  f1-score   support

          0       0.97      1.00      0.98      6339
          1       0.73      0.07      0.14       215

avg / total       0.96      0.97      0.96      6554

child_alone              precision    recall  f1-score   support

          0       1.0

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters = {
    'clf__estimator__bootstrap': [True],
    'clf__estimator__max_depth': [90, 100],
    'clf__estimator__max_features': [2, 3],
    'clf__estimator__min_samples_leaf': [4, 5],
    'clf__estimator__min_samples_split': [10, 12],
    'clf__estimator__n_estimators': [200, 300]
}

cv = GridSearchCV(pipeline, param_grid = parameters,n_jobs = -1, verbose = 2)
cv.fit(X_train,y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] clf__estimator__bootstrap=True, clf__estimator__max_depth=90, clf__estimator__max_features=2, clf__estimator__min_samples_leaf=4, clf__estimator__min_samples_split=10, clf__estimator__n_estimators=200 
[CV]  clf__estimator__bootstrap=True, clf__estimator__max_depth=90, clf__estimator__max_features=2, clf__estimator__min_samples_leaf=4, clf__estimator__min_samples_split=10, clf__estimator__n_estimators=200, total=  35.3s
[CV] clf__estimator__bootstrap=True, clf__estimator__max_depth=90, clf__estimator__max_features=2, clf__estimator__min_samples_leaf=4, clf__estimator__min_samples_split=10, clf__estimator__n_estimators=200 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   52.8s remaining:    0.0s


[CV]  clf__estimator__bootstrap=True, clf__estimator__max_depth=90, clf__estimator__max_features=2, clf__estimator__min_samples_leaf=4, clf__estimator__min_samples_split=10, clf__estimator__n_estimators=200, total=  35.2s
[CV] clf__estimator__bootstrap=True, clf__estimator__max_depth=90, clf__estimator__max_features=2, clf__estimator__min_samples_leaf=4, clf__estimator__min_samples_split=10, clf__estimator__n_estimators=200 
[CV]  clf__estimator__bootstrap=True, clf__estimator__max_depth=90, clf__estimator__max_features=2, clf__estimator__min_samples_leaf=4, clf__estimator__min_samples_split=10, clf__estimator__n_estimators=200, total=  35.6s
[CV] clf__estimator__bootstrap=True, clf__estimator__max_depth=90, clf__estimator__max_features=2, clf__estimator__min_samples_leaf=4, clf__estimator__min_samples_split=10, clf__estimator__n_estimators=300 
[CV]  clf__estimator__bootstrap=True, clf__estimator__max_depth=90, clf__estimator__max_features=2, clf__estimator__min_samples_leaf=4, clf__e

[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed: 98.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__bootstrap': [True], 'clf__estimator__max_depth': [90, 100], 'clf__estimator__max_features': [2, 3], 'clf__estimator__min_samples_leaf': [4, 5], 'clf__estimator__min_samples_split': [10, 12], 'clf__estimator__n_estimators': [200, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [16]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f753ee10048>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [25]:
print("\nBest Parameters:", cv.best_estimator_)


Best Parameters: Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
print(cv.best_estimator_)
print("The mean accuracy of the model is:",cv.score(y_test, y_pred))


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])
The mean accuracy of the model is: 0.487793713763


### 9. Export your model as a pickle file

In [2]:
import pickle
pickle.dump(pipeline,open('abc.txt', 'wb'))

NameError: name 'pipeline' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [1]:
!ls

categories.csv			   messages.csv
DisasterResponse.db		   ML Pipeline Preparation.ipynb
ETL Pipeline Preparation.ipynb	   ML Pipeline Preparation-zh.ipynb
ETL Pipeline Preparation-zh.ipynb  Twitter-sentiment-self-drive-DFE.csv
InsertDatabaseName.db
